In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle

In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip \*.zip  && rm *.zip

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/gdrive/My Drive/Kaggle/creditcard.csv", header=0)
data.head()

In [ ]:
print("Duplicates", data.duplicated().sum())
data = data.drop_duplicates()
print("Rows {}, Columns {}".format(data.shape[0], data.shape[1]))

In [ ]:
data['Class'] = data['Class'].replace({'0': 0, '1': 1})
data['Class'].unique()
data['Class'].value_counts()

In [ ]:
labels = data['Class'] 
#data = data[data.columns[:-1]]
labels.value_counts()

In [ ]:
SEED=0
X_train, X_test, y_train, y_test = train_test_split( data, labels, test_size=0.3, random_state=SEED )

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, contamination='auto')

# Fit the LOF detector on the training set
lof.fit(X_train)

In [ ]:
lof_scores_train = lof.negative_outlier_factor_
lof_threshold_train = -2.0
outliers_train = X_train[lof_scores_train < lof_threshold_train]

In [ ]:
# Remove the identified outliers from the training set
X_train_clean = X_train[lof_scores_train >= lof_threshold_train]
y_train_clean = y_train[lof_scores_train >= lof_threshold_train]

In [ ]:
gb_clf = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=SEED
)
gb_clf.fit(X_train_clean, y_train_clean)

In [ ]:
y_pred = gb_clf.predict(X_test)

In [ ]:
print("Accuracy Score: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("F1 Score: {:.2f}%".format(f1_score(y_test, y_pred) * 100))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))